# Preprocessing fMRI data with Nipype

In this notebook we will preprocess data with the standard FSL pipeline, using [Nipype](http://nipy.sourceforge.net/nipype/index.html).

Nipype is a pipelining tool that makes it easy to share pipelines of fMRI preprocessing, as well as document them, and run (them in parallel on the cluster).

First we import libraries

In [3]:
# Nipype is a pipelining tool
import nipype

# Nibabel is a library to read in nifti-files
import nibabel as nb

# Numpy is 'matlab for Python'
import numpy as np

# Matplotlib is a plotting library
import matplotlib.pyplot as plt

# Seaborn is an extension to matplotlib, that offers convenience wrappers and less ugly plots
import seaborn as sns

ImportError: No module named seaborn

We want to plot the pictures 'inline': in the notebook

In [ ]:
%matplotlib inline

### Let's load some data

In [ ]:
data = nb.load('/data/func_raw/pp0197/pp0197_B1.nii.gz').get_data()

Now 

[code]data[/code] 

is a numpy-array: a matlab-like matrix, that is efficiently stored in memory, and can be accessed fast using pointers.

1) What is the size of this matrix ([hint](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.shape.html))?

In [51]:
# ...

2) Try to average the data over the time-dimension (last-dimension; [hint](http://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html))

In [ ]:
mean_data = # ...

You can now use the following code to make an image

In [ ]:
plt.imshow(mean_data[:, 40, :,].T, origin='lower', cmap=plt.cm.gray, interpolation='nearest')
plt.axis('off')

This is an 'coronal' slice. Can you edit the code to make an _axial_ slice?

In [40]:
# ...

## Pipelining

In [42]:
# Load some pipelining libraries
from nipype.workflows.fmri.fsl import create_featreg_preproc
import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio

Now we start with the real pipelining. Crucial for this is to understand the _data structure_. Ipython notebook makes it possible to run shell ('linux') commands, like _ls_, that show the contents of a directory.

This are the directories that containt the 'raw' fMRI files.

In [43]:
!ls /data/func_raw/

pp0197	pp0381	pp0483	pp0523	pp0544	pp0549	pp0552	pp0554	pp0558
pp0372	pp0471	pp0498	pp0538	pp0548	pp0551	pp0553	pp0555	pp0559


In [44]:
!ls /data/func_raw/pp0197/

pp0197_B1.nii.gz  pp0197_B2.nii.gz


Can you draw a little sketch of how these directories look like? Can you show the contents of a different directory?

We will now set up the standard fsl preprocessing workflow, and set the smoothing to zero.

In [45]:
# Create workflow
preproc_workflow = create_featreg_preproc()


# Put the temprarory 'workflow-folder' on the big /data-disk!!
preproc_workflow.base_dir = '/data/workflow_folders'

In [48]:
# Set up smoothing and highpass filtering at 64 seconds
TR = 2.0
preproc_workflow.inputs.inputspec.fwhm = 0.0
preproc_workflow.inputs.inputspec.highpass = 64 / TR

Now we have to make a template of how the functional files look like.  Have a look at the documentation for the [SelectFiles-interface](http://nipy.sourceforge.net/nipype/interfaces/generated/nipype.interfaces.io.html#nipype-interfaces-io-selectfiles)

Can you fill in the {subject_id}-variable at the right spots?

In [49]:
templates = {'func':'/data/func_raw/ .... _B*.nii.gz'} # fill in

Let's set up a file-selector with this template

In [50]:
selector = pe.Node(nio.SelectFiles(templates), name='selector')

Now we connect the selector to the input of the preprocessing workflow

In [15]:
preproc_workflow.connect(selector, 'func', preproc_workflow.get_node("inputspec"), 'func')



4) Fill in the first 4 subject_ids

In [ ]:
selector.iterables = [('subject_id', ['0197', ...])]

In [20]:
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = '/data/preprocessed_data/'

5) We now make a datasink that collects the output of the workflow and writes it to a final directory. Can you make sure all the outputs get written to the datasink ('/data/preprocessed_data/')?

In [21]:
preproc_workflow.outputs.outputspec


highpassed_files = None
mask = None
mean = None
motion_parameters = None
motion_plots = None
realigned_files = None
reference = None
smoothed_files = None

In [22]:
preproc_workflow.connect(preproc_workflow.get_node("outputspec"), 'highpassed_files', ds, 'highpassed_files')
# ... FILL IN HERE

A nice feature of nipype is to draw a graph of a workflow. Can you make it appear? (Hint: add a .png to the dot-file, or use 'ls' to see what exactly happened)

In [ ]:
from IPython.display import Image
preproc_workflow.write_graph()

In [ ]:
Image(# FILL IN ...)

Time to run the workflow! Using the 'MultiProc'-plugin we can run the workflow in parallel on multiple cores!

In [ ]:
preproc_workflow.run(plugin='MultiProc', plugin_args={'n_procs' : 3})

Keep this window open! While the pipeline is running, you can start with the [next lesson](./part2_hddm.ipynb)